# LLM Optimization Modelling Experiment

In [661]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [749]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Generate the mathematical model

In [754]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [755]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Alkylation Process

**Objective Function:** Maximize daily profit

```
Maximize: Profit = Revenue - Cost
Profit = (Alkylate Yield * Motor Octane Number * Alkylate Price) - (Olefin Feed * Olefin Price + Isobutane Recycle * Isobutane Price + Acid Addition Rate * Acid Price + Isobutane Makeup * Isobutane Price) 
```

**Decision Variables:**

*  `Olefin Feed` (barrels per day)
*  `Isobutane Recycle` (barrels per day)
*  `Acid Addition Rate` (thousands of pounds per day)
*  `Alkylate Yield` (barrels per day)
*  `Isobutane Makeup` (barrels per day)
*  `Acid Strength` (weight percent)
*  `Motor Octane Number`
*  `External Isobutane-to-Olefin Ratio`
*  `Acid Dilution Factor`
*  `F-4 Performance Number`

**Parameters:**

*  `Alkylate Price` (dollars per barrel) -  *Assumed to be constant and known.*
*  `Olefin Price` (dollars per barrel) -  *Assumed to be constant and known.*
*  `Isobutane Price` (dollars per barrel) - *Assumed to be constant and known.*
*  `Acid Price` (dollars per thousand pounds) - *Assumed to be constant and known.*

**Regression Equations:**

*  `Alkylate Yield` = `Olefin Feed` * (1.12 + 0.13167 * `External Isobutane-to-Olefin Ratio` - 0.00667 * `External Isobutane-to-Olefin Ratio`^2) *(1 + ε1)*, where ε1 ∈ [-0.02, 0.02]
*  `Motor Octane Number` = 86.35 + 1.098 * `External Isobutane-to-Olefin Ratio` - 0.038 * `External Isobutane-to-Olefin Ratio`^2 + 0.325 * (`Acid Strength` - 89) *(1 + ε2)*, where ε2 ∈ [-0.01, 0.01]
*  `Acid Dilution Factor` = 35.82 - 0.222 * `F-4 Performance Number` *(1 + ε3)*, where ε3 ∈ [-0.05, 0.05]
*  `F-4 Performance Number` = -133 + 3 * `Motor Octane Number` *(1 + ε4)*, where ε4 ∈ [-0.1, 0.1]

**Constraints:**

*  **Variable Bounds:**  Each decision variable has a lower and upper bound (specific values need to be defined based on the process limitations).
*  **External Isobutane-to-Olefin Ratio:** `External Isobutane-to-Olefin Ratio` = (`Isobutane Recycle` + `Isobutane Makeup`) / `Olefin Feed`
*  **Acid Strength:** `Acid Strength` = (93000 * `Acid Addition Rate`) / (`Acid Addition Rate` * `Acid Dilution Factor` + 1000 * `Acid Addition Rate`)
*  **Alkylate Yield Balance:** 1.22 * `Alkylate Yield` = `Olefin Feed` + `Isobutane Makeup`

**Notes:**

* This model incorporates the regression equations with their respective error terms (ε1, ε2, ε3, ε4) to account for potential deviations from the true values. 
* The values for the parameters (Alkylate Price, Olefin Price, Isobutane Price, Acid Price) and the lower and upper bounds of the decision variables should be determined based on the specific characteristics of the alkylation process.

This mathematical optimization model can be solved using appropriate software (e.g., linear programming solvers) to find the optimal values for the decision variables that maximize the daily profit of the alkylation process. 


## 3. Generate the pyomo code

In [756]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [757]:
### Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
alkylate_price = 100  # dollars per barrel
olefin_price = 70  # dollars per barrel
isobutane_price = 60  # dollars per barrel
acid_price = 50  # dollars per thousand pounds

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.olefin_feed = pyo.Var(bounds=(1000, 5000), doc='Olefin Feed (barrels per day)')  # Sample bounds
model.isobutane_recycle = pyo.Var(bounds=(500, 2500), doc='Isobutane Recycle (barrels per day)')  # Sample bounds
model.acid_addition_rate = pyo.Var(bounds=(10, 50), doc='Acid Addition Rate (thousands of pounds per day)')  # Sample bounds
model.alkylate_yield = pyo.Var(bounds=(1000, 6000), doc='Alkylate Yield (barrels per day)')  # Sample bounds
model.isobutane_makeup = pyo.Var(bounds=(100, 500), doc='Isobutane Makeup (barrels per day)')  # Sample bounds
model.acid_strength = pyo.Var(bounds=(88, 92), doc='Acid Strength (weight percent)')  # Sample bounds
model.motor_octane_number = pyo.Var(bounds=(85, 95), doc='Motor Octane Number')  # Sample bounds
model.external_isobutane_to_olefin_ratio = pyo.Var(
    bounds=(3, 10), doc='External Isobutane-to-Olefin Ratio'
)  # Sample bounds
model.acid_dilution_factor = pyo.Var(bounds=(30, 40), doc='Acid Dilution Factor')  # Sample bounds
model.f4_performance_number = pyo.Var(bounds=(-150, -100), doc='F-4 Performance Number')  # Sample bounds

# Constraints
model.external_isobutane_to_olefin_ratio_constraint = pyo.Constraint(
    expr=model.external_isobutane_to_olefin_ratio
    == (model.isobutane_recycle + model.isobutane_makeup) / model.olefin_feed
)
model.acid_strength_constraint = pyo.Constraint(
    expr=model.acid_strength
    == (93000 * model.acid_addition_rate)
    / (model.acid_addition_rate * model.acid_dilution_factor + 1000 * model.acid_addition_rate)
)
model.alkylate_yield_balance_constraint = pyo.Constraint(
    expr=1.22 * model.alkylate_yield == model.olefin_feed + model.isobutane_makeup
)

# Regression Equations with Error Terms
model.alkylate_yield_eqn = pyo.Constraint(
    expr=model.alkylate_yield
    == model.olefin_feed
    * (
        1.12
        + 0.13167 * model.external_isobutane_to_olefin_ratio
        - 0.00667 * model.external_isobutane_to_olefin_ratio**2
    )
    * (1 + pyo.uniform(-0.02, 0.02))
)
model.motor_octane_number_eqn = pyo.Constraint(
    expr=model.motor_octane_number
    == 86.35
    + 1.098 * model.external_isobutane_to_olefin_ratio
    - 0.038 * model.external_isobutane_to_olefin_ratio**2
    + 0.325 * (model.acid_strength - 89) * (1 + pyo.uniform(-0.01, 0.01))
)
model.acid_dilution_factor_eqn = pyo.Constraint(
    expr=model.acid_dilution_factor
    == 35.82 - 0.222 * model.f4_performance_number * (1 + pyo.uniform(-0.05, 0.05))
)
model.f4_performance_number_eqn = pyo.Constraint(
    expr=model.f4_performance_number
    == -133 + 3 * model.motor_octane_number * (1 + pyo.uniform(-0.1, 0.1))
)

# Objective Function
revenue = (
    model.alkylate_yield
    * model.motor_octane_number
    * alkylate_price
)
cost = (
    model.olefin_feed * olefin_price
    + model.isobutane_recycle * isobutane_price
    + model.acid_addition_rate * acid_price
    + model.isobutane_makeup * isobutane_price
)
profit = revenue - cost

model.objective = pyo.Objective(expr=profit, sense=pyo.maximize)

# Solve
solver = pyo.SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Display results
print(results)
print("Profit = ", pyo.value(profit))
```

**Notes:**

* I have used sample data for the parameter values (prices) and the lower and upper bounds of the decision variables. You need to replace these with the actual data for your specific alkylation process.
* The `pyo.uniform(-0.02, 0.02)` parts in the regression equation constraints represent the error terms (ε1, ε2, ε3, ε4). They generate random numbers uniformly distributed between the specified bounds. 
* The solver used in this code is `ipopt`. You can use a different solver if you wish.
* This code assumes that you have Pyomo and a suitable solver installed. 


## 4. Input problem data and try running the generated code

In [758]:
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
alkylate_price = 100  # dollars per barrel
olefin_price = 70  # dollars per barrel
isobutane_price = 60  # dollars per barrel
acid_price = 50  # dollars per thousand pounds

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.olefin_feed = pyo.Var(bounds=(1000, 5000), doc='Olefin Feed (barrels per day)')  # Sample bounds
model.isobutane_recycle = pyo.Var(bounds=(500, 2500), doc='Isobutane Recycle (barrels per day)')  # Sample bounds
model.acid_addition_rate = pyo.Var(bounds=(10, 50), doc='Acid Addition Rate (thousands of pounds per day)')  # Sample bounds
model.alkylate_yield = pyo.Var(bounds=(1000, 6000), doc='Alkylate Yield (barrels per day)')  # Sample bounds
model.isobutane_makeup = pyo.Var(bounds=(100, 500), doc='Isobutane Makeup (barrels per day)')  # Sample bounds
model.acid_strength = pyo.Var(bounds=(88, 92), doc='Acid Strength (weight percent)')  # Sample bounds
model.motor_octane_number = pyo.Var(bounds=(85, 95), doc='Motor Octane Number')  # Sample bounds
model.external_isobutane_to_olefin_ratio = pyo.Var(
    bounds=(3, 10), doc='External Isobutane-to-Olefin Ratio'
)  # Sample bounds
model.acid_dilution_factor = pyo.Var(bounds=(30, 40), doc='Acid Dilution Factor')  # Sample bounds
model.f4_performance_number = pyo.Var(bounds=(-150, -100), doc='F-4 Performance Number')  # Sample bounds

# Constraints
model.external_isobutane_to_olefin_ratio_constraint = pyo.Constraint(
    expr=model.external_isobutane_to_olefin_ratio
    == (model.isobutane_recycle + model.isobutane_makeup) / model.olefin_feed
)
model.acid_strength_constraint = pyo.Constraint(
    expr=model.acid_strength
    == (93000 * model.acid_addition_rate)
    / (model.acid_addition_rate * model.acid_dilution_factor + 1000 * model.acid_addition_rate)
)
model.alkylate_yield_balance_constraint = pyo.Constraint(
    expr=1.22 * model.alkylate_yield == model.olefin_feed + model.isobutane_makeup
)

# Regression Equations with Error Terms
model.alkylate_yield_eqn = pyo.Constraint(
    expr=model.alkylate_yield
    == model.olefin_feed
    * (
        1.12
        + 0.13167 * model.external_isobutane_to_olefin_ratio
        - 0.00667 * model.external_isobutane_to_olefin_ratio**2
    )
    * (1 + pyo.uniform(-0.02, 0.02))
)
model.motor_octane_number_eqn = pyo.Constraint(
    expr=model.motor_octane_number
    == 86.35
    + 1.098 * model.external_isobutane_to_olefin_ratio
    - 0.038 * model.external_isobutane_to_olefin_ratio**2
    + 0.325 * (model.acid_strength - 89) * (1 + pyo.uniform(-0.01, 0.01))
)
model.acid_dilution_factor_eqn = pyo.Constraint(
    expr=model.acid_dilution_factor
    == 35.82 - 0.222 * model.f4_performance_number * (1 + pyo.uniform(-0.05, 0.05))
)
model.f4_performance_number_eqn = pyo.Constraint(
    expr=model.f4_performance_number
    == -133 + 3 * model.motor_octane_number * (1 + pyo.uniform(-0.1, 0.1))
)

# Objective Function
revenue = (
    model.alkylate_yield
    * model.motor_octane_number
    * alkylate_price
)
cost = (
    model.olefin_feed * olefin_price
    + model.isobutane_recycle * isobutane_price
    + model.acid_addition_rate * acid_price
    + model.isobutane_makeup * isobutane_price
)
profit = revenue - cost

model.objective = pyo.Objective(expr=profit, sense=pyo.maximize)

# Solve
solver = pyo.SolverFactory('ipopt')  # You can choose a different solver if you prefer
results = solver.solve(model)

# Display results
print(results)
print("Profit = ", pyo.value(profit))

AttributeError: module 'pyomo.environ' has no attribute 'uniform'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [759]:
print(response.text)

## Mathematical Optimization Model for Alkylation Process

**Objective Function:** Maximize daily profit

```
Maximize: Profit = Revenue - Cost
Profit = (Alkylate Yield * Motor Octane Number * Alkylate Price) - (Olefin Feed * Olefin Price + Isobutane Recycle * Isobutane Price + Acid Addition Rate * Acid Price + Isobutane Makeup * Isobutane Price) 
```

**Decision Variables:**

*  `Olefin Feed` (barrels per day)
*  `Isobutane Recycle` (barrels per day)
*  `Acid Addition Rate` (thousands of pounds per day)
*  `Alkylate Yield` (barrels per day)
*  `Isobutane Makeup` (barrels per day)
*  `Acid Strength` (weight percent)
*  `Motor Octane Number`
*  `External Isobutane-to-Olefin Ratio`
*  `Acid Dilution Factor`
*  `F-4 Performance Number`

**Parameters:**

*  `Alkylate Price` (dollars per barrel) -  *Assumed to be constant and known.*
*  `Olefin Price` (dollars per barrel) -  *Assumed to be constant and known.*
*  `Isobutane Price` (dollars per barrel) - *Assumed to be constant and known.*
*

In [761]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with actual data)
alkylate_price = 100  # dollars per barrel
olefin_price = 70  # dollars per barrel
isobutane_price = 60  # dollars per barrel
acid_price = 50  # dollars per thousand pounds

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.olefin_feed = pyo.Var(bounds=(1000, 5000), doc='Olefin Feed (barrels per day)')  # Sample bounds
model.isobutane_recycle = pyo.Var(bounds=(500, 2500), doc='Isobutane Recycle (barrels per day)')  # Sample bounds
model.acid_addition_rate = pyo.Var(bounds=(10, 50), doc='Acid Addition Rate (thousands of pounds per day)')  # Sample bounds
model.alkylate_yield = pyo.Var(bounds=(1000, 6000), doc='Alkylate Yield (barrels per day)')  # Sample bounds
model.isobutane_makeup = pyo.Var(bounds=(100, 500), doc='Isobutane Makeup (barrels per day)')  # Sample bounds
model.acid_strength = pyo.Var(bounds=(88, 92), doc='Acid Strength (weight percent)')  # Sample bounds
model.motor_octane_numbe